<a href="https://colab.research.google.com/github/elliemci/chatbots/blob/main/classifytranslate_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Chatbot that recognizes the language a user message is in and translate it English

In [2]:
!pip install dl-translate pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.6 MB/s eta 0:00:00
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215252 sha256=95d9ebe2f8a466b6678b74c7d7bdc25b91be443d9cc1bf5baa66117358eb180f
  Stored in directory: /root/.cache/pip/wheels/25/51/2d/24b40a366c85c37928d5aa36ddf257e5a79fad25e1ecd11b2c
Successfully built pyTelegramBotAPI


In [3]:
import telebot
import dl_translate as dlt

from transformers import pipeline

Get a new bot from the BotFather with /newbot in Telegram Chat, giving it a name langclasstranslate_bot and username ellismci_bot and geta HTTP API token

In [4]:
import os
from getpass import getpass

os.environ['TELEGRAM_BOT_TOKEN'] = getpass('Enter your bot token: ')
TOKEN = os.getenv('TELEGRAM_BOT_TOKEN')

# create the chatbot using pyTelegramBotAPI passing the token
bot = telebot.TeleBot(TOKEN)

Enter your bot token: ··········


In [5]:
# dl-translate is the default model,  built on top of Huggingface transformers and Facebook's mBART-Large
mt = dlt.TranslationModel(device="auto")

# instantiate the pipelineclass object with xlm-roberta-base-language-detection
langclass_pipeline = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

# xlm-roberta-base-language-detection can be used as language detector supporting 20 languages
languages = {
    "ar": "arabic",
    "bg": "bulgarian",
    "de": "german",
    "el": "modern greek",
    "en": "english",
    "es": "spanish",
    "fr": "french",
    "hi": "hindi",
    "it": "italian",
    "ja": "japanese",
    "nl": "dutch",
    "pl": "polish",
    "pt": "portuguese",
    "ru": "russian",
    "sw": "swahili",
    "th": "thai",
    "tr": "turkish",
    "ur": "urdu",
    "vi": "vietnamese",
    "zh": "chinese",
}

languages_str = ", ".join(languages.values())

# convert text into lowercase, break into words and check is a string of words containe "yes or "no"
def lowercase(text):
    return text.lower()

def capitalize(text):
    return text.capitalize()

def is_yes(text):
    words = lowercase(text).split()
    return any(word in words for word in ["yes", "please", "yep", "yeah", "y", "ya", "sure", "ok", "yup", "yep", "maybe"])

def is_no(text):
    words = lowercase(text).split()
    return any(word in words for word in ["no", "nope", "n", "nah", "not"])


# initalize bot state
state = 0
msg_lang = ""
msg_to_translate = ""

@bot.message_handler(func=lambda message: True)
def message(message):

    global state, msg_lang, msg_to_translate
    chat_id = message.chat.id

    if state == 0:
      bot.send_message(chat_id, "Hi there! Do you have a message for my to figure out its language and translete it for you?")
      state = 1

    elif state == 1:
        text = message.text

        if is_yes(text):
          bot.send_message(chat_id, "Great, please enter your message.")
          state = 2
        elif is_no(text):
          bot.send_message(chat_id, "Goodbye!")
          state = 0
          return
        else:
          bot.send_message(chat_id, "Hit me!")
          state = 2

    elif state == 2:

        text = message.text
        text_lang = langclass_pipeline(text)[0]["label"]
        proba = round(langclass_pipeline(text)[0]["score"], 2)

        if proba<0.5:
              bot.send_message(chat_id, "Sorry, your message language is not part of the 20 languages that I understand.")
              state = 5
              return

        reply = f"The message is in {languages[text_lang]} with ptobability of {proba}"
        msg_lang = capitalize(languages[text_lang])
        msg_to_translate = text
        bot.send_message(chat_id, reply)
        state = 3

    elif state == 3:
      bot.send_message(chat_id, "Do you want me to translate it for you?")
      state = 4

    elif state == 4:
      #text = message.text
      translation = mt.translate(msg_to_translate, msg_lang, 'English')
      bot.send_message(chat_id, translation)
      state = 5

    elif state == 5:
      bot.send_message(chat_id, "Do you have more foreign language messages for me to classify and translate?")
      state = 1


bot.polling()

tokenizer_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]